# Digital House - Data Science a Distancia

## Trabajo Práctico 3

### Autores: Daniel Borrino, Ivan Mongi, Jessica Polakoff, Julio Tentor
<p style="text-align:right;">Junio 2022</p>

#### Librerías necesarias

In [2]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import re
import matplotlib.pyplot as plt

In [3]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [27]:
#Levantando el csv

data = pd.read_csv('../Data/data_train.csv')

In [28]:
data.head(5)

,label,Age,Language,Sex,Marital,Has_Credit,Field,Month_of_birth,Day_of_birth,Region,Number_of_credits,Linked_cards,INPS_mln_sum,INPS_yes_no,Score_level,Score_class,Score_point,Changed_phone_number
0,0,34,1,2,6,2,13,12,1,12,1,0,0.0,0,0,0,-,1
1,0,38,1,1,5,1,10,7,1,13,1,2,0.0,0,0,0,-,1
2,0,35,1,2,4,2,9,8,1,13,4,1,1.0,1,0,0,-,1
3,0,27,1,1,5,2,13,7,1,12,1,2,1.0,0,0,0,-,1
4,0,32,1,2,4,2,10,7,1,13,3,1,2.0,1,0,0,-,1


In [29]:
data.shape

(8707, 18)

In [30]:
data.isnull().sum()

label                   0
Age                     0
Language                0
Sex                     0
Marital                 0
Has_Credit              0
Field                   0
Month_of_birth          0
Day_of_birth            0
Region                  0
Number_of_credits       0
Linked_cards            0
INPS_mln_sum            0
INPS_yes_no             0
Score_level             0
Score_class             0
Score_point             0
Changed_phone_number    0
dtype: int64

In [21]:
data.dtypes

label                     int64
Age                       int64
Language                  int64
Sex                       int64
Marital                   int64
Has_Credit                int64
Field                     int64
Month_of_birth            int64
Day_of_birth              int64
Region                    int64
Number_of_credits         int64
Linked_cards              int64
INPS_mln_sum            float64
INPS_yes_no               int64
Score_level               int64
Score_class               int64
Score_point              object
Changed_phone_number      int64
dtype: object

In [34]:
data.Score_point.value_counts()
#tiene guion del medio, lo vamos a reemplazar 

0      6571
0       590
293     198
294     156
285      85
       ... 
175       1
343       1
128       1
165       1
338       1
Name: Score_point, Length: 193, dtype: int64

In [35]:
#data['Score_point'] = data['Score_point'].replace(['-'],0)

##ver porque no me funciona y me da error :(

In [36]:
# data.dtypes()

TypeError: 'Series' object is not callable

In [37]:
#veo composicion de cada una de las variables del data set. 

data.columns

Index(['label', 'Age', 'Language', 'Sex', 'Marital', 'Has_Credit', 'Field',
       'Month_of_birth', 'Day_of_birth', 'Region', 'Number_of_credits',
       'Linked_cards', 'INPS_mln_sum', 'INPS_yes_no', 'Score_level',
       'Score_class', 'Score_point', 'Changed_phone_number'],
      dtype='object')

In [48]:
total = data.shape[0]
round(data.label.value_counts()/ total,2)

#tenemos el 92% de la variable target en defaults, quiza sea demasiado. ver! 

1    0.92
0    0.08
Name: label, dtype: float64

In [51]:
data.describe()

,label,Age,Language,Sex,Marital,Has_Credit,Field,Month_of_birth,Day_of_birth,Region,Number_of_credits,Linked_cards,INPS_mln_sum,INPS_yes_no,Score_level,Score_class,Changed_phone_number
count,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000,8707.000000
mean,0.923050,37.690479,1.147468,1.622488,4.540829,1.032043,10.485012,6.360859,1.462157,11.207190,1.119789,1.100838,2.132932,0.957505,0.547491,1.082807,0.112094
std,0.266527,10.750031,0.354592,0.484792,1.053161,0.176125,5.597442,3.405988,0.498594,3.020191,0.366915,0.315294,2.225331,0.201726,1.190702,2.447644,0.344394
min,0.000000,19.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,29.000000,1.000000,1.000000,4.000000,1.000000,6.000000,3.000000,1.000000,12.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,1.000000,36.000000,1.000000,2.000000,4.000000,1.000000,13.000000,6.000000,1.000000,12.000000,1.000000,1.000000,1.700000,1.000000,0.000000,0.000000,0.000000
75%,1.000000,44.000000,1.000000,2.000000,5.000000,1.000000,14.000000,9.000000,2.000000,13.000000,1.000000,1.000000,2.800000,1.000000,0.000000,0.000000,0.000000
max,1.000000,79.000000,2.000000,2.000000,7.000000,2.000000,22.000000,12.000000,2.000000,14.000000,5.000000,4.000000,86.300000,1.000000,4.000000,11.000000,3.000000
